##### Copyright 2025 Google LLC.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Multimodal Live API - Quickstart

<a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/Get_started_LiveAPI.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" height=30/></a>

This notebook demonstrates simple usage of the Gemini 2.0 Multimodal Live API. For an overview of new capabilities refer to the [Gemini 2.0 docs](https://ai.google.dev/gemini-api/docs/models/gemini-v2).

This notebook implements a simple turn-based chat where you send messages as text, and the model replies with audio. The API is capable of much more than that. The goal here is to demonstrate with **simple code**.

Some features of the API are not working in Colab, to try them it is recommended to have a look at this [python script](./Get_started_LiveAPI.py) and run it locally.

If you aren't looking for code, and just want to try multimedia streaming use [Live API in Google AI Studio](https://aistudio.google.com/app/live).

The [Next steps](#next_steps) section at the end of this tutorial provides links to additional resources.

## Setup

### Install SDK

The new **[Google Gen AI SDK](https://ai.google.dev/gemini-api/docs/sdks)** provides programmatic access to Gemini 2.0 (and previous models) using both the [Google AI for Developers](https://ai.google.dev/gemini-api/docs) and [Vertex AI](https://cloud.google.com/vertex-ai/generative-ai/docs/overview) APIs. With a few exceptions, code that runs on one platform will run on both.

More details about this new SDK on the [documentation](https://ai.google.dev/gemini-api/docs/sdks) or in the [Getting started](../quickstarts/Get_started.ipynb) notebook.

In [1]:
%pip install -U -q google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 3.3 MB/s eta 0:00:00


### Set up your API key

To run the following cell, your API key must be stored in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see [Authentication](../quickstarts/Authentication.ipynb) for an example.

In [2]:
from google.colab import userdata
import os

os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

### Initialize SDK client

The client will pick up your API key from the environment variable.
To use the live API you need to set the client version to `v1alpha`.

In [18]:
from google import genai
from google.genai import types
client = genai.Client(http_options= {'api_version': 'v1beta'})

### Select a model

Multimodal Live API are a new capability introduced with the [Gemini 2.0](https://ai.google.dev/gemini-api/docs/models/gemini-v2) model. It won't work with previous generation models.


In [19]:
MODEL = "gemini-2.0-flash-live-001"

### Import

Import all the necessary modules.

In [20]:
import asyncio
import base64
import contextlib
import datetime
import os
import json
import wave
import itertools

from IPython.display import display, Audio

from google import genai
from google.genai import types

## Text to Text

The simplest way to use the Live API is as a text-to-text chat interface, but it can do **a lot** more than this.

In [21]:
last_handle = None

config={
    "response_modalities": ["TEXT"]
}

async with client.aio.live.connect(model=MODEL, config=config) as session:
  message = "Hello? Gemini are you there?"
  print("> ", message, "\n")
  await session.send(input=message, end_of_turn=True)

  # For text responses, When the model's turn is complete it breaks out of the loop.
  turn = session.receive()
  async for chunk in turn:
    if chunk.text is not None:
      print(f'- {chunk.text}')

>  Hello? Gemini are you there? 



<ipython-input-21-144979044fc5>:10: DeprecationWarning: The `session.send` method is deprecated and will be removed in a future version (not before Q3 2025).
Please use one of the more specific methods: `send_client_content`, `send_realtime_input`, or `send_tool_response` instead.
  await session.send(input=message, end_of_turn=True)


- Yes, I am
-  here! How can I help you today?



## Simple text to audio

The simplest way to playback the audio in Colab, is to write it out to a `.wav` file. So here is a simple wave file writer:

In [22]:
@contextlib.contextmanager
def wave_file(filename, channels=1, rate=24000, sample_width=2):
    with wave.open(filename, "wb") as wf:
        wf.setnchannels(channels)
        wf.setsampwidth(sample_width)
        wf.setframerate(rate)
        yield wf

The next step is to tell the model to return audio by setting `"response_modalities": ["AUDIO"]` in the `LiveConnectConfig`.  

When you get a response from the model, then you write out the data to a `.wav` file.

In [23]:
config={
    "response_modalities": ["AUDIO"]}

async def async_enumerate(aiterable):
  n=0
  async for item in aiterable:
    yield n, item
    n+=1


async with client.aio.live.connect(model=MODEL, config=config) as session:
  file_name = 'audio.wav'
  with wave_file(file_name) as wav:
    message = "Hello? Gemini are you there?"
    print("> ", message, "\n")
    await session.send(input=message, end_of_turn=True)

    turn = session.receive()
    async for n,response in async_enumerate(turn):
      if response.data is not None:
        wav.writeframes(response.data)

        if n==0:
          print(response.server_content.model_turn.parts[0].inline_data.mime_type)
        print('.', end='')


display(Audio(file_name, autoplay=True))


>  Hello? Gemini are you there? 



<ipython-input-23-d1fbaacf1d50>:16: DeprecationWarning: The `session.send` method is deprecated and will be removed in a future version (not before Q3 2025).
Please use one of the more specific methods: `send_client_content`, `send_realtime_input`, or `send_tool_response` instead.
  await session.send(input=message, end_of_turn=True)


audio/pcm;rate=24000
............

## Towards Async Tasks


The real power of the Live API is that it's real time, and interruptable. You can't get that full power in a simple sequence of steps. To really use the functionality you will move the `send` and `recieve` operations (and others) into their own [async tasks](https://docs.python.org/3/library/asyncio-task.html).

Because of the limitations of Colab this tutorial doesn't totally implement the interactive async tasks, but it does implement the next step in that direction:

- It separates the `send` and `receive`, but still runs them sequentially.  
- In the next tutorial you'll run these in separate `async` tasks.


Setup a quick logger to make debugging easier (switch to `setLevel('DEBUG')` to see debugging messages).

In [24]:
import logging

logger = logging.getLogger('Live')
logger.setLevel('INFO')

The class below implements the interaction with the Live API.

In [25]:
class AudioLoop:
  def __init__(self, turns=None,  config=None):
    self.session = None
    self.index = 0
    self.turns = turns
    if config is None:
      config={
          "response_modalities": ["AUDIO"]}
    self.config = config

  async def run(self):
    logger.debug('connect')
    async with client.aio.live.connect(model=MODEL, config=self.config) as session:
      self.session = session

      async for sent in self.send():
        # Ideally send and recv would be separate tasks.
        await self.recv()

  async def _iter(self):
    if self.turns:
      for text in self.turns:
        print("message >", text)
        yield text
    else:
      print("Type 'q' to quit")
      while True:
        text = await asyncio.to_thread(input, "message > ")

        # If the input returns 'q' quit.
        if text.lower() == 'q':
          break

        yield text

  async def send(self):
    async for text in self._iter():
      logger.debug('send')

      # Send the message to the model.
      await self.session.send(input=text, end_of_turn=True)
      logger.debug('sent')
      yield text

  async def recv(self):
    # Start a new `.wav` file.
    file_name = f"audio_{self.index}.wav"
    with wave_file(file_name) as wav:
      self.index += 1

      logger.debug('receive')

      # Read chunks from the socket.
      turn = self.session.receive()
      async for n, response in async_enumerate(turn):
        logger.debug(f'got chunk: {str(response)}')

        if response.data is None:
          logger.debug(f'Unhandled server message! - {response}')
        else:
          wav.writeframes(response.data)
          if n == 0:
            print(response.server_content.model_turn.parts[0].inline_data.mime_type)
          print('.', end='')

      print('\n<Turn complete>')

    display(Audio(file_name, autoplay=True))
    await asyncio.sleep(2)


There are 3 methods worth describing here:

**`run` - The main loop**

This method:

- Opens a `websocket` connecting to the Live API.
- Calls the initial `setup` method.
- Then enters the main loop where it alternates between `send` and `recv` until send returns `False`.
- The next tutorial will demonstrate how to stream media and run these asynchronously.

**`send` - Sends input text to the api**

The `send` method collects input text from the user, wraps it in a `client_content` message (an instance of `BidiGenerateContentClientContent`), and sends it to the model.

If the user sends a `q` this method returns `False` to signal that it's time to quit.

**`recv` - Collects audio from the API and plays it**

The `recv` method collects audio chunks in a loop and writes them to a `.wav` file. It breaks out of the loop once the model sends a `turn_complete` method, and then plays the audio.

To keep things simple in Colab it collects **all** the audio before playing it. [Other examples](#next_steps) demonstrate how to play audio as soon as you start to receive it (using `PyAudio`), and how to interrupt the model (implement input and audio playback on separate tasks).

### Run

Run it:


In [26]:
await AudioLoop(['Hello', "What's your name?"]).run()

message > Hello


<ipython-input-25-e8f5fd55e512>:41: DeprecationWarning: The `session.send` method is deprecated and will be removed in a future version (not before Q3 2025).
Please use one of the more specific methods: `send_client_content`, `send_realtime_input`, or `send_tool_response` instead.
  await self.session.send(input=text, end_of_turn=True)


audio/pcm;rate=24000
........
<Turn complete>


message > What's your name?
audio/pcm;rate=24000
....................
<Turn complete>


## Working with resumable sessions

Session resumption allows you to return to a previous interaction with the Live API by sending the last session handle you got from the previous session.

When you set your session to be resumable, the session information keeps stored on the Live API for up to 24 hours. In this time window, you can resume the conversation and refer to previous information you have shared with the model.


### Helper functions

Start by creating the helper functions for your resumable interaction with the Live API. It will include:

In [29]:
import asyncio
import traceback
from asyncio.exceptions import CancelledError

last_handle = None

MODEL = "gemini-2.0-flash-live-001"

client = genai.Client(http_options= {'api_version': 'v1beta'})

async def async_enumerate(aiterable):
  n=0
  async for item in aiterable:
    yield n, item
    n+=1


def show_response(response):
    new_handle = None
    if text := response.text:
        print(text, end="")
    else:
      print(response.model_dump_json(indent=2, exclude_none=True))
    if response.session_resumption_update:
        new_handle = response.session_resumption_update.new_handle
    return new_handle


async def clock():
  time = 0
  while True:
    await asyncio.sleep(60)
    time += 1
    print(f"{time}:00")


async def recv(session):
  global last_handle
  try:
    while True:
        async for response in session.receive():
            new_handle = show_response(response)
            if new_handle:
                last_handle = new_handle
  except asyncio.CancelledError:
    pass


async def send(session):
  while True:
      message = await asyncio.to_thread(input, "message > ")
      if message.lower() == "q":
          break
      await session.send_client_content(turns={
          'role': 'user',
          'parts': [{'text': message}]
      })


async def async_main(last_handle=None):
  config = types.LiveConnectConfig.model_validate({
      "response_modalities": ["TEXT"],
      "session_resumption": {
          'handle': last_handle,
      }
  })
  try:
    async with (
        client.aio.live.connect(model=MODEL, config=config) as session,
        asyncio.TaskGroup() as tg
    ):
        clock_task = tg.create_task(clock())
        recv_task = tg.create_task(recv(session))
        send_task = tg.create_task(send(session))
        await send_task
        raise asyncio.CancelledError()
  except asyncio.CancelledError:
      pass
  except ExceptionGroup as EG:
      traceback.print_exception(EG)

Now you can start interacting with the Live API (type `q` to finish the conversation):

In [30]:
await async_main()

message > how are you doing Today?
{
  "session_resumption_update": {}
}
I am doing well, thank you for asking! As a large language model, I don't experience emotions or feelings like humans do, but I am functioning optimally and ready to assist you. How can I help you today?
{
  "server_content": {
    "generation_complete": true
  }
}
{
  "server_content": {
    "turn_complete": true
  },
  "usage_metadata": {
    "prompt_token_count": 14,
    "response_token_count": 47,
    "total_token_count": 61,
    "prompt_tokens_details": [
      {
        "modality": "TEXT",
        "token_count": 14
      }
    ],
    "response_tokens_details": [
      {
        "modality": "TEXT",
        "token_count": 47
      }
    ]
  }
}
{
  "session_resumption_update": {
    "new_handle": "Cgxra2o3bHc1bWVmaG8",
    "resumable": true
  }
}
message > what is the capital of Brazil?
{
  "session_resumption_update": {}
}
The capital of Brazil is **Brasília**.
{
  "server_content": {
    "generation_complete

With the session resumption you have the session handle to refer to your previous sessions. In this example, the handle is saved at the `last_handle` variable as below:

In [31]:
last_handle

'Cgwxc3M1OXdvYmY1bHA'

Now you can start a new Live API session, but this time pointing to a handle from a previous session. Also, to test you could gather information from the previous session, you will ask the model what was the second question you asked before (in this example, it was "what is the capital of Brazil?"). You can see the Live API recovering that information:

In [32]:
await async_main(last_handle)

message > hello there!
{
  "session_resumption_update": {}
}
Hello to you too! How can I help you today?
{
  "server_content": {
    "generation_complete": true
  }
}
{
  "server_content": {
    "turn_complete": true
  },
  "usage_metadata": {
    "prompt_token_count": 101,
    "response_token_count": 13,
    "total_token_count": 114,
    "prompt_tokens_details": [
      {
        "modality": "TEXT",
        "token_count": 101
      }
    ],
    "response_tokens_details": [
      {
        "modality": "TEXT",
        "token_count": 13
      }
    ]
  }
}
{
  "session_resumption_update": {
    "new_handle": "CgxxeG54YW93Y2sxeXk",
    "resumable": true
  }
}
message > what was the second question I asked?
{
  "session_resumption_update": {}
}
The second question you asked me was: "what is the capital of Brazil?"
{
  "server_content": {
    "generation_complete": true
  }
}
{
  "server_content": {
    "turn_complete": true
  },
  "usage_metadata": {
    "prompt_token_count": 132,
    "res

## Next steps

<a name="next_steps"></a>

This tutorial just shows basic usage of the Live API, using the Python GenAI SDK.

- If you aren't looking for code, and just want to try multimedia streaming use [Live API in Google AI Studio](https://aistudio.google.com/app/live).
- If you want to see how to setup streaming interruptible audio and video using the Live API see the [Audio and Video input Tutorial](../quickstarts/Get_started_LiveAPI.py).
- If you're interested in the low level details of using the websockets directly, see the [websocket version of this tutorial](../quickstarts/quickstarts/websockets/Get_started_LiveAPI.ipynb).
- Try the [Tool use in the live API tutorial](../quickstarts/Get_started_LiveAPI_tools.ipynb) for an walkthrough of Gemini-2's new tool use capabilities.
- There is a [Streaming audio in Colab example](../quickstarts/quickstarts/websockets/live_api_streaming_in_colab.ipynb), but this is more of a **demo**, it's **not optimized for readability**.
- Other nice Gemini 2.0 examples can also be found in the [Cookbook's 2.0 directory](https://github.com/google-gemini/cookbook/tree/main/gemini-2/), in particular the [video understanding](../quickstarts/Video_understanding.ipynb) and the [spatial understanding](../quickstarts/Spatial_understanding.ipynb) ones.
